In [ ]:
# 安装工具链
%pip install uv
!uv tool install "huggingface_hub[cli]"


In [ ]:
# ✅ Step 1: 获取最近n个提交
import requests
import ipywidgets as widgets
from IPython.display import display
import subprocess

# 仓库信息
repo = "index-tts/index-tts"
commits_url = f"https://api.github.com/repos/{repo}/commits"
response = requests.get(commits_url)
commits = response.json()

# 提取前两个提交 SHA 和信息
options = []
for commit in commits[:3]:
    sha = commit["sha"]
    message = commit["commit"]["message"]
    options.append((f"{message[:60]}... ({sha[:7]})", sha))  # 显示前60字+SHA前7位

# ✅ Step 2: 创建下拉菜单和按钮
dropdown = widgets.Dropdown(
    options=options,
    description='选择版本:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

button = widgets.Button(description="下载选定版本", button_style='success')
output = widgets.Output()

# ✅ Step 3: 下载并解压所选版本
def download_selected_version(sha):
    url = f"https://github.com/index-tts/index-tts/archive/{sha}.zip"
    subprocess.run(["wget", "-O", "main.zip", url])
    subprocess.run(["unzip", "main.zip"])
    subprocess.run(["mv", f"index-tts-{sha}", "index-tts"])
    subprocess.run(["rm", "main.zip"])

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"正在下载版本：{dropdown.label}")
        download_selected_version(dropdown.value)
        print("✅ 下载完成，已解压到 index-tts 文件夹")

button.on_click(on_button_clicked)

# ✅ Step 4: 显示控件
display(dropdown, button, output)


In [ ]:
# 安装依赖
!cd /content/index-tts/ && uv sync --all-extras

In [ ]:
# 下载模型 checkpoints
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

In [37]:
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts


In [ ]:
# 启动

%cd /content/index-tts/
!uv run webui_with_share.py